In [1]:
import sys

sys.path.append('/Users/charlesdedampierre/Desktop/bunka_related_projects/BunkaTopics')

In [2]:
from bunkatopics import Bunka
from langchain.embeddings import HuggingFaceEmbeddings
import random
from datasets import load_dataset
import os
from dotenv import load_dotenv

load_dotenv()

True

In [9]:
import sys

sys.path.append("../")

import sqlite3

import os
import pandas as pd

from dotenv import load_dotenv

# Load the environment variables from the .env file
dotenv_path = os.path.join("../", ".env")
load_dotenv(dotenv_path)


conn = sqlite3.connect(os.getenv("DB_SCIENCE_PATH"))

df = pd.read_sql("SELECT * FROM wiki_abtract", conn)
df = df.drop_duplicates('wikidata_id', keep='first')

In [10]:
texts = list(df.clean_abstract)
ids =  list(df.wikidata_id)

In [15]:
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5")

bunka = Bunka(embedding_model=embedding_model)
bunka.fit(docs = texts, ids = ids)

INFO      |2023-11-08 18:07:57|{}|fit|Extracting Terms
/Users/charlesdedampierre/Desktop/bunka_related_projects/BunkaTopics/.venv/lib/python3.10/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.5.0) was trained with spaCy v3.5.0 and may not be 100% compatible with the current version (3.6.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
100%|██████████████████████████████████████████████████| 12794/12794 [03:52<00:00, 54.99it/s]
INFO      |2023-11-08 18:11:52|{}|fit|Embedding Documents, this may take few minutes
INFO      |2023-11-08 18:22:21|{}|fit|Reducing Dimensions
/Users/charlesdedampierre/Desktop/bunka_related_projects/BunkaTopics/.venv/lib/python3.10/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed

In [48]:
# Topic Modeling
df_topics = bunka.get_topics(n_clusters=20, name_lenght=4, min_count_terms = 50, top_terms_overall=3000)

In [49]:
list(df_topics.name)

['economist | economics | philosopher | economy',
 'footballer | cricketer | player | politician',
 'astronomer | philosopher | astrologer | mathematician',
 'physicist | electricity | inventor | heat',
 'mineralogist | geologist | paleontologist | mineralogy',
 'naturalist | botanist | illustrator | plants',
 'anatomist | anatomy | physiologist | physiology',
 'polymath | astronomy | commentary | astronomers',
 'archaeologist | excavations | archaeology | naturalist',
 'entomologist | entomology | insects | ornithologist',
 'astronomer | mathematician | mathematics | geometry',
 'histoire | naturelle | entomologique | Muséum',
 'archaeologist | geologist | geology | palaeontologist',
 'cartographer | maps | sco | map',
 'ethnographer | team | statesman | commander',
 'botanist | genus | plant | botany',
 'composer | music | theorist | composers',
 'dynasty | period | official | calendar',
 'chemist | chemistry | chemical | acid',
 'cartographer | maps | geographer | archaeologist']

In [44]:
?bunka.get_topics

Signature:
bunka.get_topics(
    n_clusters=5,
    ngrams=[1, 2],
    name_lenght=15,
    top_terms_overall=2000,
    min_count_terms=20,
) -> pandas.core.frame.DataFrame
Docstring: <no docstring>
File:      ~/Desktop/bunka_related_projects/BunkaTopics/bunkatopics/bunkatopics.py
Type:      method

In [24]:
df_docs = pd.DataFrame([x.dict() for x in bunka.docs])

In [27]:
df_topics = pd.DataFrame([x.dict() for x in bunka.topics])

In [29]:
df_bunka = pd.merge(df_docs, df_topics, on = 'topic_id')

In [31]:
df_bunka = df_bunka[['doc_id', 'content', 'topic_id', 'name']]

In [35]:
df_bunka = df_bunka[~df_bunka['content'].str.contains('may refer to')]
df_bunka = df_bunka[~df_bunka['content'].str.contains('could refer')]

In [43]:
list(set(df_bunka.name))

['astronomer | asteroid | observatory | crater | comets | asteroids | comet | Moon | planet | astronomy',
 'footballer | player | cricketer | driver | union | John | politician | producer | antiquary | actor',
 'team | medal | bronze | event | ethnographer | Imperial | ice | commander | hockey | chairman',
 'entomologist | insects | ornithologist | entomology | species | insect | für | collection | birds | Diptera',
 'cartographer | chart | conquest | cartographers | navigator | charts | maps | islands | map | atlases',
 'Carl Linnaeus | botanist | Uppsala University | plants | abbreviation | zoologist | reptiles | Java | genus | plant',
 'pharmacist | medicine | zoologist | physician | biologist | pharmacy | doctorate | zoology | professor | physiologist',
 'astronomer | mathematician | Royal Society | Isaac Newton | telescope | fellow | clergyman | astronomy | equation | Oxford',
 'cricketer | footballer | football | politician | coach | judge | basketball | player | actor | sailor',